## ALS Recommendation model on explicit movie rating dataset known as [MovieLens](https://grouplens.org/datasets/movielens/100k/) for movie recommendation 

---
Library Importing

In [9]:
import numpy as np 
import pandas as pd 
import scipy

from recommenders.datasets import movielens

from implicit.als import AlternatingLeastSquares

---
Data Loading & prep.

In [19]:
data = movielens.load_pandas_df(
    size='100k',
    header=['UserId', 'MovieId', 'Rating', 'Timestamp'],
    title_col='Title'
)

data.loc[:, 'Rating'] = data['Rating'].astype(np.float32)
data.head()

100%|██████████| 4.81k/4.81k [00:11<00:00, 432KB/s]  


,UserId,MovieId,Rating,Timestamp,Title
0,196,242,3.0,881250949,Kolya (1996)
1,63,242,3.0,875747190,Kolya (1996)
2,226,242,5.0,883888671,Kolya (1996)
3,154,242,3.0,879138235,Kolya (1996)
4,306,242,5.0,876503793,Kolya (1996)


In [51]:
movies = pd.Series(data['Title'].values, index=data['MovieId'].values).drop_duplicates()
movies.sort_index(inplace=True)
print(movies)

1                                Toy Story (1995)
2                                GoldenEye (1995)
3                               Four Rooms (1995)
4                               Get Shorty (1995)
5                                  Copycat (1995)
                          ...                    
1677                         Sweet Nothing (1995)
1678                            Mat' i syn (1997)
1679                             B. Monkey (1998)
1681                          You So Crazy (1994)
1682    Scream of Stone (Schrei aus Stein) (1991)
Length: 1664, dtype: object


In [73]:
movie_arr = np.array(movies.values)
print(movie_arr)

['Toy Story (1995)' 'GoldenEye (1995)' 'Four Rooms (1995)' ...
 'B. Monkey (1998)' 'You So Crazy (1994)'
 'Scream of Stone (Schrei aus Stein) (1991)']


In [41]:
sparse_df = data.loc[:, ['UserId', 'MovieId', 'Rating']].sort_values(['UserId', 'MovieId'])
row_ind = sparse_df['UserId'].to_numpy()
col_ind = sparse_df['MovieId'].to_numpy()
data_ = sparse_df['Rating'].to_numpy()
# inds = list(zip(row_ind, col_ind))
# data_arr = list(zip(data_, inds))

sparse_mat = scipy.sparse.csr_matrix((data_, (row_ind, col_ind)), shape=(len(row_ind), len(col_ind)), dtype=np.int32)

In [42]:
print(type(sparse_mat))
print(sparse_mat[34])

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 242)	5
  (0, 245)	4
  (0, 259)	2
  (0, 286)	5
  (0, 288)	2
  (0, 289)	1
  (0, 292)	5
  (0, 294)	1
  (0, 299)	5
  (0, 310)	4
  (0, 312)	4
  (0, 324)	5
  (0, 329)	5
  (0, 332)	5
  (0, 690)	4
  (0, 898)	5
  (0, 899)	5
  (0, 990)	5
  (0, 991)	4
  (0, 1024)	5


In [43]:
user_csr = sparse_mat.tocsr()

---
Model Training

In [44]:
model = AlternatingLeastSquares(factors=64, regularization=0.05)
model.fit(user_csr)

/Users/nipunpathitage/opt/anaconda3/envs/rec/lib/python3.7/site-packages/implicit/utils.py:27: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "
100%|██████████| 15/15 [00:01<00:00, 12.56it/s]


---
Model Evaluation & Prediction

In [74]:
user = 54
ids, scores = model.recommend(user, user_csr[user], N=30, filter_already_liked_items=False)
pd.DataFrame({"artist": movie_arr[ids], "score": scores, "already_liked": np.in1d(ids, user_csr[user].indices)})

,artist,score,already_liked
0,L.A. Confidential (1997),1.254556,True
1,"Cable Guy, The (1996)",1.235535,True
2,Fly Away Home (1996),1.114977,False
3,Chasing Amy (1997),1.090906,True
4,Twister (1996),1.063261,True
5,Kingpin (1996),1.055062,True
6,Maya Lin: A Strong Clear Vision (1994),1.035456,True
7,"Apostle, The (1997)",1.024874,True
8,Babe (1995),1.011662,True
9,Absolute Power (1997),1.010076,True


In [75]:
ids, scores = model.recommend(user, user_csr[user], N=10, filter_already_liked_items=True)
pd.DataFrame({"artist": movie_arr[ids], "score": scores, "already_liked": np.in1d(ids, user_csr[user].indices)})

,artist,score,already_liked
0,Fly Away Home (1996),1.114977,False
1,"Promesse, La (1996)",0.801016,False
2,Richard III (1995),0.736884,False
3,Cop Land (1997),0.661183,False
4,Sling Blade (1996),0.623124,False
5,Turbo: A Power Rangers Movie (1997),0.617109,False
6,Devil in a Blue Dress (1995),0.602197,False
7,Evita (1996),0.579314,False
8,"Crossing Guard, The (1995)",0.576421,False
9,"Bloody Child, The (1996)",0.573901,False
